In [ ]:
# Restful API Script

from flask import Flask, request, redirect, render_template
from flask_caching import Cache
from flask_uploads import UploadSet, configure_uploads, ALL
import os
import shutil
import glob


cache = Cache(config={'CACHE_TYPE': 'null'})
app = Flask(__name__)
cache.init_app(app)
files = UploadSet('files', ALL)
app.config['UPLOADED_FILES_DEST'] = './static/uploaded_imgs'
configure_uploads(app, files)
OUTPUT_PATH = './static/result_imgs/detected'+'detected.jpg'
roots_to_clear = [app.config['UPLOADED_FILES_DEST'], './static/result_imgs']


@app.route('/', methods=['GET', 'POST'])
def index():
    for root_to_clear in roots_to_clear:
        for root, dirs, files in os.walk(root_to_clear):
            for file in files:
                os.remove(os.path.join(root, file))
            for dir in dirs:
                shutil.rmtree(os.path.join(root, dir))
    return render_template('home.html')


@app.route('/preview', methods=['GET', 'POST'])
def upload_and_preview():
    if request.method == 'POST' and 'media' in request.files:
        _ = files.save(request.files['media'])
        list_of_files = glob.glob(os.path.join(app.config['UPLOADED_FILES_DEST'], '*'))
        latest_file = max(list_of_files, key=os.path.getctime)
        global image_to_process
        image_to_process = latest_file
        img_to_render = os.path.join('..', latest_file)
        img_to_render = img_to_render.replace('\\', '/')
    else:
        img_to_render = '../static/images/no_image.png'
    paths = ['../static/images/no_image.png']

    return render_template('preview.html', img_name=img_to_render, img_paths=paths)


@app.route('/preview/detection', methods=['GET', 'POST'])
def obj_detect():

    if request.method == 'POST':
        from imageai.Detection import ObjectDetection
        model = ObjectDetection()
        model.setModelTypeAsYOLOv3()
        model.setModelPath("./models/yolo.h5")
        model.loadModel()
        detections, paths = model.detectObjectsFromImage(input_image=image_to_process,
                                                         output_image_path=OUTPUT_PATH,
                                                         extract_detected_objects=True)
        img_to_render = os.path.join('..', image_to_process)
        img_to_render = img_to_render.replace('\\', '/')
        paths = [os.path.join('..', path) for path in paths]
        paths = [path.replace('\\', '/') for path in paths]
        for i, path in enumerate(paths):
            detections[i]['index'] = i
            detections[i]['path'] = path
        for detection in detections:
            detection['percentage_probability'] = round(detection['percentage_probability'], 2)
        return render_template('result.html', img_name=img_to_render, img_details=detections, num_obj=len(detections))


@app.after_request
def add_header(r):
    """
    Add headers to both force latest IE rendering engine or Chrome Frame,
    and also to cache the rendered page for 10 minutes.
    """
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r


if __name__ == '__main__':
    # init()
    app.run()















 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\Sriram Saran Cheruvu\.conda\envs\tensorflow\lib\site-packages\flask_caching\__init__.py:192: UserWarning: Flask-Caching: CACHE_TYPE is set to null, caching is effectively disabled.
  "Flask-Caching: CACHE_TYPE is set to null, "
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2020 13:46:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:46:40] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:46:40] "GET /static/images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:46:40] "GET /static/images/no_image.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:46:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jun/2020 13:47:48] "POST /preview HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:47:48] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:47:48] "GET /static/images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:47:48] "GET /static/uploaded_imgs/traffic.jpg HTTP/1.1" 200 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


127.0.0.1 - - [22/Jun/2020 13:48:30] "POST /preview/detection HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /static/result_imgs/detecteddetected.jpg-objects/truck-1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /static/result_imgs/detecteddetected.jpg-objects/truck-2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /static/result_imgs/detecteddetected.jpg-objects/truck-3.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /static/result_imgs/detecteddetected.jpg-objects/truck-4.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /preview/static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [22/Jun/2020 13:48:30] "GET /static/images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2020 13:48:31] "GET /static/result_imgs/detecteddetected.jpg-objects/bus-5.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:31] "GET /static/result_imgs/detecteddetected.jpg-objects/bus-6.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2020 13:48:31] "GE